In [1]:
%pip install --quiet openai python-dotenv pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Load environment and libraries
from dotenv import load_dotenv
import os
import io
import base64
import json
from PIL import Image

# OpenAI client (adjust import based on your openai library version)
try:
    from openai import OpenAI
except Exception as e:
    OpenAI = None

# Load .env and create a client if available
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_KEY) if (OpenAI and OPENAI_KEY) else None

if client is None:
    print("OpenAI client not initialized. Please set OPENAI_API_KEY in your environment or .env file.")
else:
    print("OpenAI client ready.")

OpenAI client ready.


In [3]:
# Utility: encode image to base64 (JPEG)
def encode_image_to_base64(image_path: str) -> str:
    """Open a given image path and return a base64-encoded JPEG string.
    Returns: base64 string (without data: prefix).
    """
    img = Image.open(image_path).convert('RGB')
    buf = io.BytesIO()
    img.save(buf, format='JPEG')
    return base64.b64encode(buf.getvalue()).decode('utf-8')

# Small heuristic fallback classifier (very simple)
def heuristic_classify(image_path: str) -> dict:
    """Try a few cheap heuristics when the API is unavailable:
    - If filename contains 'receipt' or OCR says 'total', classify as 'receipt'
    - If OCR contains medical keywords, 'health'
    - Otherwise 'other'
    """
    category = 'other'
    explanation = 'Fallback heuristics used.
    conf = 0.5

    # Check filename
    name = os.path.basename(image_path).lower()
    if any(k in name for k in ['receipt','invoice','bill']):
        return {"category": 'receipt', "confidence": 0.6, "explanation": 'Filename looks like a receipt'}

    # Try lightweight OCR if pytesseract is installed
    try:
        import pytesseract
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img).lower()
        if any(k in text for k in ['total', 'subtotal', 'receipt', 'amount']):
            return {"category": 'receipt', "confidence": 0.8, "explanation": 'OCR indicates receipt-like text (e.g. Total)'}
        if any(k in text for k in ['prescription', 'rx', 'prescribed', 'clinic', 'hospital', 'patient']):
            return {"category": 'health', "confidence": 0.7, "explanation": 'OCR indicates medical words'}
    except Exception:
        # pytesseract not available or some error - just ignore
        pass

    return {"category": category, "confidence": conf, "explanation": 'No strong heuristics matched; defaulting to other.'}

SyntaxError: unterminated string literal (detected at line 19) (2173001363.py, line 19)

In [ ]:
# The main classify function: uses OpenAI if available, otherwise falls back
def classify_image(image_path: str, allowed_categories=None, model='gpt-4o') -> dict:
    if allowed_categories is None:
        allowed_categories = ['scenery', 'food', 'receipt', 'health', 'collectable', 'other']

    # First: try heuristic if no client
    if client is None:
        print('No OpenAI client available; using heuristic fallback')
        return heuristic_classify(image_path)

    # Build the prompt for the expert agent. We ask for strict JSON to parse easily
    system_prompt = (
        'You are an expert image classification assistant. The allowed output categories are: ' + ', '.join(allowed_categories) + 

: 
,
: { 
: 
 },
: [